## 2022/01/24 (mon)
### 1. desktop.ini 제거하고 resize 완료하기 (train, test)
### 2. function로 변경 (image load, image excel export, image resize)
### 3. data shuffle() using dataset-proportion.xlsx
- https://helloyjam.github.io/tensorflow/buffer-size-in-shuffle/

## 1/25
### - 경로에 한글 포함 자제
### - 절대 경로보다는 상대 경로로

### batch : 아이템을 N개의 그룹으로 묶는다
### shuffle을 사용하기 위해서는 먼저 _tensor_slices()사용하여 tf.Tensor로 변환
### -> 이를 활용하기 위해서는 CSV, NUMPY 데이터 필요

In [1]:
import os
import csv
import PIL
import glob
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from PIL import Image
from skimage.transform import resize
import matplotlib.pyplot as plt

In [2]:
!cd # current directory
print(os.getcwd())

C:\Users\jinse\source\python\CNN-transportation-classfication


지정된 경로를 찾을 수 없습니다.


In [3]:
path = "C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day" 
file_list = os.listdir(path)

unknown_index = file_list.index('Unknown')
print(unknown_index)

new_list_wo_unknown = file_list[:unknown_index] + file_list[unknown_index+1:]
print(new_list_wo_unknown) # 실제로 사용할 리스트

unknown_list = file_list[unknown_index]
print(unknown_list) # ?

6
['Bus', 'Light', 'Motorcycle', 'Sedan', 'SUV', 'Truck', 'Van']
Unknown


In [4]:
path = "C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day"

# same structure in new_list_wo_unknown
dirs = os.listdir(path + "/" + new_list_wo_unknown[0]) # representative, 질문
print(dirs)

# new_list_wo_unknown: ['Bus', 'Light', 'Motorcycle', 'Sedan', 'SUV', 'Truck', 'Van']
# dirs: ['Easy', 'Hard', 'Moderate']

# except unknown
images = [] # 왜 각각의 클래스가 아닌 images 리스트에 다 모으는지
label = []
for file in new_list_wo_unknown:
    dir_path = path + "/" + file

    for d in dirs: # 질문
        new_dir_path = dir_path + "/" + d # ex) bus -> easy
        os_list = os.listdir(new_dir_path)
        for os_file in os_list:
            full_name = new_dir_path + '/' + os_file
            # print(full_name)
            # os.listdir = 지정한 디렉토리 내의 모든 파일과 디렉토리의 리스트를 리턴
            images.append(full_name) # images 리스트에 new_dir_path리스트의 파일들을 추가하여 연장
        # images 리스트에 bus->easy 파일이 들어가 있는 상태
        for i in range(len(os_list)):
            label.append(file) # bus의 easy카테고리에 속한 파일들을 label에 추가

# unknown
unknown_dir_path = path + "/" + unknown_list
os_list = os.listdir(unknown_dir_path)
for os_file in os_list:
    images.append(unknown_dir_path + '/' + os_file)

for i in range(len(os_list)):
    label.append(unknown_list)
    

print(len(images))
print(len(label))

print(images)
print(label)

['Easy', 'Hard', 'Moderate']
9386
9386
['C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/0PpIHLBv_1.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/1oQNZhnH_2.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/2n0TfHrm_1.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/2NrbKI4G_2.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/3jD0k5uN_1.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/3mIOIQvq_0.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/3ro6exbt_0.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/48y5R2jt_1.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/4eyq4hTc_1.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/4SwxdDUB_1.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/7zYnUK7x_1.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/877Y17LX_2.jpg', 'C:/Users/jinse/OneDrive

In [5]:
file_list

['Bus', 'Light', 'Motorcycle', 'Sedan', 'SUV', 'Truck', 'Unknown', 'Van']

In [6]:
# i = element in list
for file in file_list: 
    # i = bus, light, motorcycle ... (index in hidden)
    print(file)
print()

# i == index
for i in range(len(file_list)):
    print(file_list[i])

Bus
Light
Motorcycle
Sedan
SUV
Truck
Unknown
Van

Bus
Light
Motorcycle
Sedan
SUV
Truck
Unknown
Van


In [7]:
"""
1. 왜 엑셀로 작업하는 과정이 필요한 것인가?

"""

'\n1. 왜 엑셀로 작업하는 과정이 필요한 것인가?\n\n'

### 데이터 프레임을 엑셀로 저장
1. 데이터 프레임 생성
2. 먼저 차종별로 데이터를 나눠야 함



In [8]:
data = {'images' : images, 'label' : label}
data # 딕셔너리 생성?, 여기서 인덱스가 필요?

{'images': ['C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/0PpIHLBv_1.jpg',
  'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/1oQNZhnH_2.jpg',
  'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/2n0TfHrm_1.jpg',
  'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/2NrbKI4G_2.jpg',
  'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/3jD0k5uN_1.jpg',
  'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/3mIOIQvq_0.jpg',
  'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/3ro6exbt_0.jpg',
  'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/48y5R2jt_1.jpg',
  'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/4eyq4hTc_1.jpg',
  'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/4SwxdDUB_1.jpg',
  'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/7zYnUK7x_1.jpg',
  'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/877Y17LX_2.jpg',
  'C:/Users/jinse/OneDrive/바탕 

In [22]:
data = pd.DataFrame(data) # Unknown이 왜 포함이 되었는지
data
print(data.shape)
print(data.to_numpy()) # label 또한 numpy로 변하게끔

(9386, 2)
[['C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/0PpIHLBv_1.jpg'
  'Bus']
 ['C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/1oQNZhnH_2.jpg'
  'Bus']
 ['C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/2n0TfHrm_1.jpg'
  'Bus']
 ...
 ['C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Unknown/Xqe7ZEqf_2.jpg'
  'Unknown']
 ['C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Unknown/y0Vzd1bG_1.jpg'
  'Unknown']
 ['C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Unknown/YTKiY90L_2.jpg'
  'Unknown']]


In [10]:
data.to_excel('C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/train.xlsx') # unknown파일 포함? 

In [11]:
path = "C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day"
file_list_Test = os.listdir(path)

unknown_index_Test = file_list_Test.index('Unknown')
print(unknown_index_Test)

new_list_wo_unknown_Test = file_list_Test[:unknown_index_Test] + file_list_Test[unknown_index_Test+1:]
print(new_list_wo_unknown_Test) # 실제로 사용할 리스트

unknown_list_Test = file_list_Test[unknown_index_Test]
print(unknown_list_Test) # ?

6
['Bus', 'Light', 'Motorcycle', 'Sedan', 'SUV', 'Truck', 'Van']
Unknown


In [12]:
path = "C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day"

# same structure in new_list_wo_unknown
dirs = os.listdir(path + "/" + new_list_wo_unknown_Test[0]) # representative, 질문
print(dirs)

# new_list_wo_unknown: ['Bus', 'Light', 'Motorcycle', 'Sedan', 'SUV', 'Truck', 'Van']
# dirs: ['Easy', 'Hard', 'Moderate']

# except unknown
images_Test = [] # 왜 각각의 클래스가 아닌 images 리스트에 다 모으는지
label_Test = []
for file in new_list_wo_unknown_Test:
    dir_path = path + "/" + file

    for d in dirs: # 질문
        new_dir_path = dir_path + "/" + d # ex) bus -> easy
        os_list_Test = os.listdir(new_dir_path)
        for os_file in os_list_Test:
            full_name = new_dir_path + '/' + os_file
            images_Test.append(full_name)
        # os.listdir = 지정한 디렉토리 내의 모든 파일과 디렉토리의 리스트를 리턴
        
        # images 리스트에 bus->easy 파일이 들어가 있는 상태
        for i in range(len(os_list_Test)):
            label_Test.append(file) # bus의 easy카테고리에 속한 파일들을 label에 추가

# unknown
unknown_dir_path = path + "/" + unknown_list_Test
os_list_Test = list(set(os.listdir(unknown_dir_path)) - {'desktop.ini', 'whatever.ini'})
for f in os_list_Test:
    images_Test.append(unknown_dir_path + "/" + f)


for i in range(len(os_list_Test)):
    label_Test.append(unknown_list_Test)
    

print(len(images_Test))
print(len(label_Test))

print(images_Test)
print(label_Test)

['Easy', 'Hard', 'Moderate']
4602
4602
['C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Easy/002226_2.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Easy/002227_2.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Easy/002228_2.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Easy/002229_2.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Easy/002350_1.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Easy/002387_2.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Easy/002390_1.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Easy/002723_0.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Hard/001241_3.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Moderate/001935_0.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Moderate/002225_1.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Moderate/002225_2.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/D

In [13]:
data_Test = {'images' : images_Test, 'label' : label_Test}
data_Test # 딕셔너리 생성?, 여기서 인덱스가 필요?

{'images': ['C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Easy/002226_2.jpg',
  'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Easy/002227_2.jpg',
  'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Easy/002228_2.jpg',
  'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Easy/002229_2.jpg',
  'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Easy/002350_1.jpg',
  'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Easy/002387_2.jpg',
  'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Easy/002390_1.jpg',
  'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Easy/002723_0.jpg',
  'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Hard/001241_3.jpg',
  'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Moderate/001935_0.jpg',
  'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Moderate/002225_1.jpg',
  'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Moderate/002225_2.jpg',
  'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/B

In [14]:
data_Test = pd.DataFrame(data_Test) # Unknown이 왜 포함이 되었는지
data_Test

images    label
0     C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/...      Bus
1     C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/...      Bus
2     C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/...      Bus
3     C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/...      Bus
4     C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/...      Bus
...                                                 ...      ...
4597  C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/...  Unknown
4598  C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/...  Unknown
4599  C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/...  Unknown
4600  C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/...  Unknown
4601  C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/...  Unknown

[4602 rows x 2 columns]

In [15]:
data_Test.to_excel('C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/test.xlsx') # unknown파일 포함? 

# data resize and resolution check
- we select 64 x 64 ok!!!

In [16]:
# train set files
print(images)
print(label)

sep = images[0].index("작업중_220104") 
path = 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Resize'

for f in images:
    img = Image.open(f, 'r') # read mode
    resize_img = img.resize((64, 64))
    
    full_name = path + f[sep+10:]
    dir_name, file_name = os.path.split(full_name)
    os.makedirs(dir_name, exist_ok=True)
    resize_img.save(full_name)
    

['C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/0PpIHLBv_1.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/1oQNZhnH_2.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/2n0TfHrm_1.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/2NrbKI4G_2.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/3jD0k5uN_1.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/3mIOIQvq_0.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/3ro6exbt_0.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/48y5R2jt_1.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/4eyq4hTc_1.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/4SwxdDUB_1.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/7zYnUK7x_1.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/877Y17LX_2.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day/Bus/Easy/94

In [17]:
# test set files
print(images_Test)
print(label_Test)

sep = images_Test[0].index("작업중_220104") 
path = 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Resize'

for f in images_Test:
    img = Image.open(f, 'r') # read mode
    resize_img = img.resize((64, 64))
    
    full_name = path + f[sep+10:]
    dir_name, file_name = os.path.split(full_name)
    os.makedirs(dir_name, exist_ok=True)
    resize_img.save(full_name)

['C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Easy/002226_2.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Easy/002227_2.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Easy/002228_2.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Easy/002229_2.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Easy/002350_1.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Easy/002387_2.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Easy/002390_1.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Easy/002723_0.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Hard/001241_3.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Moderate/001935_0.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Moderate/002225_1.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Moderate/002225_2.jpg', 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Bus/Moderate/002226_1.jpg', 'C:/User

UnidentifiedImageError: cannot identify image file 'C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day/Motorcycle/Easy/desktop.ini'

In [ ]:
PIL.Image.open(str(images[0]))

In [ ]:
# resize

# 사이즈를 조정하게 된다면 화질이 깨지는 경우 발생?
path = "C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Test/Day"

for file in new_list_wo_unknown_Test:
    dir_path = path + "/" + file

    for d in dirs: # 질문
        new_dir_path = dir_path + "/" + d # ex) bus -> easy
        os_list_Test = os.listdir(new_dir_path)
        for os_file in os_list_Test:
            full_name = new_dir_path + '/' + os_file
            images_Test.append(full_name)
        # os.listdir = 지정한 디렉토리 내의 모든 파일과 디렉토리의 리스트를 리턴
        
        # images 리스트에 bus->easy 파일이 들어가 있는 상태
        for i in range(len(os_list_Test)):
            label_Test.append(file) # bus의 easy카테고리에 속한 파일들을 label에 추가


In [ ]:
for file in new_list_wo_unknown_Test:
    dir_path = path + "/" + file

    for d in dirs: # 질문
        new_dir_path = dir_path + "/" + d # ex) bus -> easy
        os_list_Test = os.listdir(new_dir_path)
        for os_file in os_list_Test:
            full_name = new_dir_path + '/' + os_file
            images_Test.append(full_name)
        # os.listdir = 지정한 디렉토리 내의 모든 파일과 디렉토리의 리스트를 리턴
        
        # images 리스트에 bus->easy 파일이 들어가 있는 상태
        for i in range(len(os_list_Test)):
            label_Test.append(file) # bus의 easy카테고리에 속한 파일들을 label에 추가


In [ ]:
path = "C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day"

# same structure in new_list_wo_unknown
dirs = os.listdir(path + "/" + new_list_wo_unknown[0]) # representative, 질문
print(dirs)

# new_list_wo_unknown: ['Bus', 'Light', 'Motorcycle', 'Sedan', 'SUV', 'Truck', 'Van']
# dirs: ['Easy', 'Hard', 'Moderate']

# except unknown
images = [] # 왜 각각의 클래스가 아닌 images 리스트에 다 모으는지
label = []
for file in new_list_wo_unknown:
    dir_path = path + "/" + file

    for d in dirs: # 질문
        new_dir_path = dir_path + "/" + d # ex) bus -> easy
        os_list = os.listdir(new_dir_path)
        for os_file in os_list:
            full_name = new_dir_path + '/' + os_file
            # print(full_name)
            # os.listdir = 지정한 디렉토리 내의 모든 파일과 디렉토리의 리스트를 리턴
            images.append(full_name) # images 리스트에 new_dir_path리스트의 파일들을 추가하여 연장
        # images 리스트에 bus->easy 파일이 들어가 있는 상태
        for i in range(len(os_list)):
            label.append(file) # bus의 easy카테고리에 속한 파일들을 label에 추가

# unknown
unknown_dir_path = path + "/" + unknown_list
os_list = os.listdir(unknown_dir_path)
images.extend(os_list)

for i in range(len(os_list)):
    label.append(unknown_list)
    

print(len(images))
print(len(label))

print(images)
print(label)

In [ ]:
# resize

path = "C:/Users/jinse/OneDrive/바탕 화면/작업중_220104/Train/Day"

# 이미지 크기 지정 
image_w = 64 
image_h = 64

# 이미지 데이터 읽어 들이기 --- (※3)
images = []
label = []
nb_classes = len(new_list_wo_unknown)

for idx, category in enumerate(path):
    # 레이블 지정 --- (※4)
    files = os.listdir(path + '/' + category)
    
    for f in files:
        img = Image.open(path + '/' + category + '/' + )
    label = [0 for i in range(nb_classes)]
    label[idx] = 1

for file in new_list_wo_unknown:
    dir_path = path + "/" + file

    for d in dirs: # 질문
        new_dir_path = dir_path + "/" + d # ex) bus -> easy
        os_list = os.listdir(new_dir_path)
        for os_file in os_list:
            full_name = new_dir_path + '/' + os_file
            img = Image.open(new_dir_path + '/' + os_file + '/')
            # print(full_name)
            # os.listdir = 지정한 디렉토리 내의 모든 파일과 디렉토리의 리스트를 리턴
            resize_img = img.resize((64, 64))



In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))